<a href="https://colab.research.google.com/github/pankajti/wqu/blob/main/capstone/module4/breakout_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
## import list of all NSE tickers 

In [62]:
import pandas as pd 
import yfinance as yf
import datetime as dt
import sqlite3
import sqlalchemy
import os

In [63]:
start = (dt.datetime.now()-dt.timedelta(6) ).strftime('%Y-%m-%d')
current_s = dt.datetime.now().strftime('%Y%m%d')

In [64]:
# list of all companies from https://www.nseindia.com/regulations/listing-compliance/nse-market-capitalisation-all-companies

In [65]:
tickers_path = '/Users/pankajti/dev/git/wqu/capstone/module4/MCAP31122023_0.xlsx'
tickers_data = pd.read_excel(tickers_path,nrows=2189)

In [66]:
yahoo_symbs=tickers_data.Symbol+'.NS'

In [67]:
yahoo_symbs= yahoo_symbs.dropna().tolist()

In [68]:
sum(pd.isna(yahoo_symbs))

0

In [69]:
len(yahoo_symbs)

2189

In [70]:
#minute_data_1000 = yf.Tickers(yahoo_symbs[:1000]).history(start=start, interval ='1m')

In [71]:
all_yf_data = []
failed_tickers = []
for t in yahoo_symbs:
    try:
        yf_data = yf.download(t,start=start, interval ='1m')
        yf_data['ticker']=t
        all_yf_data.append(yf_data)
    except Exception as e :
        print(f"error while downloading for {t}")
        failed_tickers.append(t)

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['RELIANCE.NS']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TCS.NS']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['HDFCBANK.NS']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ICICIBANK.NS']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))
[*********************100%%**********************]  1 of 1 completed
[**********

In [72]:
minute_data= pd.concat(all_yf_data)

In [73]:
minute_data.head()

,Open,High,Low,Close,Adj Close,Volume,ticker
2024-04-22 09:15:00+05:30,1414.699951,1418.199951,1413.500000,1418.199951,1418.199951,0.0,INFY.NS
2024-04-22 09:16:00+05:30,1418.849976,1423.349976,1417.500000,1418.099976,1418.099976,182792.0,INFY.NS
2024-04-22 09:17:00+05:30,1418.050049,1421.000000,1416.900024,1418.949951,1418.949951,47858.0,INFY.NS
2024-04-22 09:18:00+05:30,1419.349976,1419.349976,1416.550049,1416.650024,1416.650024,52072.0,INFY.NS
2024-04-22 09:19:00+05:30,1417.449951,1417.650024,1413.949951,1413.949951,1413.949951,47266.0,INFY.NS


## Load data to DB

In [80]:
db_data_path = r'/Users/pankajti/dev/git/wqu/capstone/data/db/capstone.db'
con = sqlite3.connect(db_data_path)
dbEngine=sqlalchemy.create_engine(f'sqlite:///{db_data_path}')

#files = os.listdir(datafiles_path)
try:
    loaded_dates =list(pd.read_sql("select date from market_data",dbEngine)['date'].unique())
except:
    loaded_dates=[]

#files=['w']


file_data = minute_data.reset_index()
#file_data = pd.read_csv(market_data_path )

file_data=file_data.rename({'index':'timestamp'}, axis =1)
#file_data['date'] = file_data.timestamp.apply(lambda x:x[:10])
file_data['date'] = file_data.timestamp.apply(lambda x:x.strftime('%Y-%m-%d'))

file_data=file_data[~file_data.date.isin(loaded_dates)]
if len(file_data)>100:
    file_data.to_sql(con=dbEngine, name = 'market_data', index=False, if_exists='append')
    print(f"loaded {file_data.shape} records  for {f}")
    loaded_dates.extend(file_data.date.unique())

else:
    print(f"data already exists for {f}")
print(f"done for {f}")



NameError: name 'f' is not defined

In [77]:
minute_data.reset_index()

,index,Open,High,Low,Close,Adj Close,Volume,ticker
0,2024-04-22 09:15:00+05:30,1414.699951,1418.199951,1413.500000,1418.199951,1418.199951,0.0,INFY.NS
1,2024-04-22 09:16:00+05:30,1418.849976,1423.349976,1417.500000,1418.099976,1418.099976,182792.0,INFY.NS
2,2024-04-22 09:17:00+05:30,1418.050049,1421.000000,1416.900024,1418.949951,1418.949951,47858.0,INFY.NS
3,2024-04-22 09:18:00+05:30,1419.349976,1419.349976,1416.550049,1416.650024,1416.650024,52072.0,INFY.NS
4,2024-04-22 09:19:00+05:30,1417.449951,1417.650024,1413.949951,1413.949951,1413.949951,47266.0,INFY.NS
...,...,...,...,...,...,...,...,...
1896209,2024-04-25 10:45:00+05:30,4.450000,4.450000,4.450000,4.450000,4.450000,20.0,LAKPRE.NS
1896210,2024-04-25 11:01:00+05:30,4.450000,4.450000,4.450000,4.450000,4.450000,300.0,LAKPRE.NS
1896211,2024-04-25 12:04:00+05:30,4.450000,4.450000,4.450000,4.450000,4.450000,1435.0,LAKPRE.NS
1896212,2024-04-25 12:05:00+05:30,4.450000,4.450000,4.450000,4.450000,4.450000,0.0,LAKPRE.NS


## Save zip file

In [ ]:
minute_data.to_csv(f'/Users/pankajti/dev/git/wqu/capstone/data/yf_ohlc/minute_data_{current_s}.csv.zip',compression='zip')